<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/Stagecoach-Project/blob/main/Fetching_Data_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests pandas


In [ ]:
import requests
import pandas as pd
import time
from IPython.display import clear_output


In [ ]:
# Fetch user inputs
line_id = input("Enter the line id: ")
stop_point_id = input("Enter the stop point id: ")
direction = input("Enter the direction (inbound/outbound): ")

Enter the line id: 22
Enter the stop point id: 490000173RF
Enter the direction (inbound/outbound): inbound


In [ ]:
def fetch_arrival_predictions(line_id, stop_point_id, direction=None, destination_station_id=None):
    base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals"

    params = {
        'naptanId': stop_point_id,
    }
    if direction:
        params['direction'] = direction
    if destination_station_id:
        params['destinationStationId'] = destination_station_id

    response = requests.get(base_url, params=params)
    data = response.json()

    # Parse the data
    predictions = []
    for item in data:
        predictions.append({
            'Line': item['lineName'],
            'Destination': item['destinationName'],
            'Expected Arrival': item['expectedArrival'],
            'Platform': item.get('platformName', 'N/A'),
            'Current Location': item.get('currentLocation', 'N/A'),
        })

    # Display the data in a readable form using pandas
    df = pd.DataFrame(predictions)
    return df

# Refresh the data every 15 seconds
while True:
    df = fetch_arrival_predictions(line_id, stop_point_id, direction)
    clear_output(wait=True)
    print(df)
    time.sleep(15)  # Refresh every 15 seconds

In [ ]:
def fetch_arrival_predictions(line_id, stop_point_id, direction=None, destination_station_id=None):
    base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals"

    params = {
        'naptanId': stop_point_id,
    }
    if direction:
        params['direction'] = direction
    if destination_station_id:
        params['destinationStationId'] = destination_station_id

    response = requests.get(base_url, params=params)
    data = response.json()

    # Parse the data
    predictions = []
    for item in data:
        predictions.append({
            'Line ID': item['lineId'],
            'Vehicle ID': item['vehicleId'],
            'Stop Point ID': item['naptanId'],
            'Station Name': item['stationName'],
            'Direction': item['direction'],
            'Expected Arrival': item['expectedArrival'],
        })

    # Display the data in a readable form using pandas
    df = pd.DataFrame(predictions)
    return df
while True:
    df = fetch_arrival_predictions(line_id, stop_point_id, direction)
    clear_output(wait=True)
    print(df)
    time.sleep(15)

Streaming output truncated to the last 5000 lines.
182  2024-06-20T14:08:10Z  
183  2024-06-20T14:26:29Z  
184  2024-06-20T14:29:03Z  

[185 rows x 6 columns]
    Line ID Vehicle ID Stop Point ID                Station Name Direction  \
0        22    BV66VHY   490014150KA              Walpole Street   inbound   
1        22    YY66OZG    490014186I      Wandsworth Bridge Road  outbound   
2        22    BV66VHY    490006224G         Duke of York Square   inbound   
3        22    LJ62KHV    490000206H       Sloane Square Station   inbound   
4        22    BV66ZRZ    490005138N               Wilton Street  outbound   
..      ...        ...           ...                         ...       ...   
180      22    BV66VJU   490000093PA          Green Park Station  outbound   
181      22    BV66VHY    490000119P    Hyde Park Corner Station   inbound   
182      22    YY66OYW    490011281N               Putney Common  outbound   
183      22    YY66OZG    490005076E  Hobury Street / Worlds 

21/06/2024



In [2]:
!pip install pandas requests pytz

import pandas as pd
import requests
import time
import pytz
from datetime import datetime
from IPython.display import clear_output

# TfL API details
api_url = 'https://api.tfl.gov.uk/StopPoint/490008660N/Arrivals'
headers = {'accept': 'application/json'}

# Define time zones
gmt = pytz.timezone('GMT')
bst = pytz.timezone('Europe/London')

def fetch_bus_data():
    response = requests.get(api_url, headers=headers)
    data = response.json()
    df = pd.json_normalize(data)

    # Convert times from GMT to BST
    df['expectedArrival'] = pd.to_datetime(df['expectedArrival']).dt.tz_convert(bst)
    df['scheduledArrival'] = pd.to_datetime(df['scheduledArrival']).dt.tz_convert(bst)

    # Adjust to local time (laptop time)
    df['expectedArrival'] = df['expectedArrival'].dt.tz_localize(None)
    df['scheduledArrival'] = df['scheduledArrival'].dt.tz_localize(None)

    return df

def calculate_ewt(df):
    df['ewt'] = (df['expectedArrival'] - df['scheduledArrival']).dt.total_seconds() / 60.0
    return df

while True:
    # Clear the previous output
    clear_output(wait=True)

    # Fetch and process data
    df = fetch_bus_data()
    df = calculate_ewt(df)

    # Display the latest data
    print(df[['lineId', 'vehicleId', 'expectedArrival', 'scheduledArrival', 'ewt']])

    # Wait for 30 seconds before fetching data again
    time.sleep(30)


KeyError: 'scheduledArrival'

In [3]:
!pip install pandas requests pytz

import pandas as pd
import requests
import time
import pytz
from datetime import datetime
from IPython.display import clear_output

# TfL API details
api_url = 'https://api.tfl.gov.uk/StopPoint/490008660N/Arrivals'
headers = {'accept': 'application/json'}

# Define time zones
gmt = pytz.timezone('GMT')
bst = pytz.timezone('Europe/London')

def fetch_bus_data():
    response = requests.get(api_url, headers=headers)
    data = response.json()
    df = pd.json_normalize(data)

    # Check if 'expectedArrival' and 'scheduledArrival' columns are present
    if 'expectedArrival' in df.columns and 'scheduledArrival' in df.columns:
        # Convert times from GMT to BST
        df['expectedArrival'] = pd.to_datetime(df['expectedArrival']).dt.tz_convert(bst)
        df['scheduledArrival'] = pd.to_datetime(df['scheduledArrival']).dt.tz_convert(bst)

        # Adjust to local time (laptop time)
        df['expectedArrival'] = df['expectedArrival'].dt.tz_localize(None)
        df['scheduledArrival'] = df['scheduledArrival'].dt.tz_localize(None)

        return df
    else:
        print("Expected columns are not present in the data.")
        return pd.DataFrame()  # Return an empty DataFrame if the columns are missing

def calculate_ewt(df):
    if not df.empty:
        df['ewt'] = (df['expectedArrival'] - df['scheduledArrival']).dt.total_seconds() / 60.0
    return df

while True:
    # Clear the previous output
    clear_output(wait=True)

    # Fetch and process data
    df = fetch_bus_data()
    if not df.empty:
        df = calculate_ewt(df)
        # Display the latest data
        print(df[['lineId', 'vehicleId', 'expectedArrival', 'scheduledArrival', 'ewt']])
    else:
        print("No data to display.")

    # Wait for 30 seconds before fetching data again
    time.sleep(30)



Expected columns are not present in the data.
No data to display.


KeyboardInterrupt: 